# Interactive Loan Demo

Simple loan visualization with time machine.

In [1]:
import sys
import os
from datetime import datetime, timedelta
from decimal import Decimal

sys.path.append(os.path.dirname(os.getcwd()))

from money_warp import Money, InterestRate, Loan, Warp
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go

# Global variables
current_loan = None
time_dates = []

# Widgets
principal_input = widgets.FloatText(value=10000, description='Principal ($):')
rate_input = widgets.Text(value='5% annual', description='Rate:')
payments_input = widgets.IntSlider(value=12, min=1, max=60, description='Payments:')
amortization_input = widgets.Dropdown(
    options=['PriceScheduler', 'InvertedPriceScheduler'],
    value='PriceScheduler',
    description='Amortization:'
)
fine_rate_input = widgets.FloatText(value=2.0, description='Fine Rate (%):')
create_button = widgets.Button(description='Create Loan', button_style='primary')

time_slider = widgets.IntSlider(value=0, min=0, max=100, description='Time Machine:', disabled=True)
payment_input = widgets.FloatText(value=0, description='Payment ($):')
record_button = widgets.Button(description='Record Payment', button_style='success')

# Outputs
loan_output = widgets.Output()
balance_output = widgets.Output()
chart_output = widgets.Output()

print('✅ Setup complete!')

✅ Setup complete!


In [2]:
def create_loan(b):
    global current_loan, time_dates
    with loan_output:
        loan_output.clear_output()
        principal = Money(str(principal_input.value))
        rate = InterestRate(rate_input.value)
        
        # Get scheduler class
        from money_warp.scheduler import PriceScheduler, InvertedPriceScheduler
        scheduler_map = {
            'PriceScheduler': PriceScheduler,
            'InvertedPriceScheduler': InvertedPriceScheduler
        }
        scheduler = scheduler_map[amortization_input.value]
        
        # Convert fine rate from percentage to decimal
        fine_rate = Decimal(str(fine_rate_input.value / 100))
        
        start_date = datetime.now().replace(day=1) + timedelta(days=32)
        start_date = start_date.replace(day=1)
        
        due_dates = []
        for i in range(payments_input.value):
            month = start_date.month + i
            year = start_date.year + (month - 1) // 12
            month = ((month - 1) % 12) + 1
            due_dates.append(datetime(year, month, start_date.day))
        
        current_loan = Loan(
            principal, 
            rate, 
            due_dates,
            scheduler=scheduler,
            fine_rate=fine_rate
        )
        
        # Time machine setup - DAILY steps
        start = current_loan.disbursement_date
        end = max(due_dates) + timedelta(days=150)
        time_dates = []
        current = start
        while current <= end:
            time_dates.append(current)
            current += timedelta(days=1)  # 1 day increments
        
        time_slider.max = len(time_dates) - 1
        time_slider.disabled = False
        
        print(f'🎉 ${principal.real_amount:,.2f} loan at {rate}')
        print(f'📊 Amortization: {amortization_input.value}')
        print(f'🚨 Fine Rate: {fine_rate_input.value}% per missed payment')
        
        # Show amortization schedule with payment buttons
        schedule = current_loan.get_amortization_schedule()
        print(f'\n📋 Amortization Schedule ({len(schedule)} payments):')
        
        # Create payment buttons for each installment
        global payment_buttons
        payment_buttons = []
        
        for i, entry in enumerate(schedule):
            # Create button for this installment
            button = widgets.Button(
                description=f'Pay #{i+1}',
                button_style='info',
                layout=widgets.Layout(width='80px')
            )
            
            # Create a closure to capture the due date
            def make_payment_handler(due_date):
                def handler(b):
                    # Calculate the current expected payment amount (may be higher due to accrued interest)
                    with Warp(current_loan, due_date) as warped_loan:
                        expected_payment = warped_loan.get_expected_payment_amount(due_date)
                    
                    current_loan.record_payment(expected_payment, due_date)
                    
                    with loan_output:
                        print(f'✅ Installment payment: ${expected_payment.real_amount:,.2f} on {due_date.strftime("%Y-%m-%d")}')
                    update_display()
                return handler
            
            button.on_click(make_payment_handler(entry.due_date))
            payment_buttons.append(button)
            
            # Display the schedule line with button
            display(widgets.HBox([
                button,
                widgets.HTML(f'<div style="padding: 5px; font-family: monospace;">'
                           f'{i+1:2d}. {entry.due_date.strftime("%Y-%m-%d")} | '
                           f'${entry.payment_amount.real_amount:>8,.2f} | '
                           f'Interest: ${entry.interest_payment.real_amount:>6,.2f} | '
                           f'Principal: ${entry.principal_payment.real_amount:>6,.2f} | '
                           f'Balance: ${entry.ending_balance.real_amount:>8,.2f}'
                           f'</div>')
            ]))
        
        update_display()

def record_payment(b):
    if current_loan and payment_input.value > 0:
        amount = Money(str(payment_input.value))
        # Record payment at current time machine position
        current_time = time_dates[time_slider.value]
        current_loan.record_payment(amount, current_time)
        
        with loan_output:
            print(f"✅ Payment of ${amount.real_amount:,.2f} recorded on {current_time.strftime('%Y-%m-%d')}")
        
        payment_input.value = 0
        update_display()

def update_display():
    if not current_loan:
        return
        
    current_time = time_dates[time_slider.value]
    
    with balance_output:
        balance_output.clear_output()
        with Warp(current_loan, current_time) as warped_loan:
            balance = warped_loan.current_balance
            fines = warped_loan.outstanding_fines
        
        # Use clean loan methods
        principal_balance = warped_loan.principal_balance
        accrued_interest = warped_loan.accrued_interest
        total_owed = warped_loan.current_balance
        days = warped_loan.days_since_last_payment(current_time)
        
        print(f'📅 {current_time.strftime("%Y-%m-%d")} | 💰 Actual Balance: ${total_owed.real_amount:,.2f}')
        print(f'🏦 Principal Balance: ${principal_balance.real_amount:,.2f}')
        if accrued_interest.real_amount > 0:
            print(f'📈 Accrued Interest: ${accrued_interest.real_amount:,.2f} ({days} days)')
        if fines.real_amount > 0:
            print(f'🚨 Outstanding Fines: ${fines.real_amount:,.2f}')
        
        # Show complete entries history (payments + fines + interest)
        all_entries = []
        
        # Add all payments
        for payment in warped_loan._all_payments:
            if payment.datetime <= current_time:
                category = getattr(payment, 'category', 'payment')
                if category == 'actual_fine':
                    desc_text = '(Fine payment)'
                elif category == 'actual_interest':
                    desc_text = '(Interest payment)'
                elif category == 'actual_principal':
                    desc_text = '(Principal payment)'
                else:
                    desc_text = '(Payment)'
                
                all_entries.append({
                    'date': payment.datetime,
                    'amount': payment.amount.real_amount,
                    'description': desc_text,
                    'type': 'payment'
                })
        
        # Add fine applications
        for due_date, fine_amount in warped_loan.fines_applied.items():
            if due_date <= current_time:
                all_entries.append({
                    'date': due_date,
                    'amount': fine_amount.real_amount,
                    'description': '(Fine applied)',
                    'type': 'fine'
                })
        
        # Sort all entries by date
        all_entries.sort(key=lambda x: x['date'])
        
        if all_entries:
            print(f'\n📋 Entries History ({len(all_entries)} entries):')
            for i, entry in enumerate(all_entries, 1):
                if entry['type'] == 'fine':
                    print(f'  {i}. {entry["date"].strftime("%Y-%m-%d")}: +${entry["amount"]:,.2f} {entry["description"]} 🚨')
                else:
                    print(f'  {i}. {entry["date"].strftime("%Y-%m-%d")}: -${entry["amount"]:,.2f} {entry["description"]} 💳')
    
    with chart_output:
        chart_output.clear_output(wait=True)
        
        schedule = current_loan.get_amortization_schedule()
        expected_dates = [current_loan.disbursement_date] + [entry.due_date for entry in schedule]
        expected = [float(current_loan.principal.real_amount)] + [float(entry.ending_balance.real_amount) for entry in schedule]
        
        # Generate daily dates from start to current time
        start_date = current_loan.disbursement_date
        actual_dates, actual_balances, principal_balances, interest_balances, actual_fines = [], [], [], [], []
        
        current_date = start_date
        while current_date <= current_time:
            with Warp(current_loan, current_date) as warped_loan:
                # Use clean loan methods
                principal_bal = warped_loan.principal_balance
                accrued_int = warped_loan.accrued_interest
                fines_only = warped_loan.outstanding_fines
                total_bal = warped_loan.current_balance
                
                actual_dates.append(current_date)
                actual_balances.append(float(total_bal.real_amount))
                principal_balances.append(float(principal_bal.real_amount))
                interest_balances.append(float(accrued_int.real_amount))
                actual_fines.append(float(fines_only.real_amount))
            
            current_date += timedelta(days=1)  # Daily intervals for precision
        
        fig = go.Figure()
        
        # Expected balance (dashed line)
        fig.add_trace(go.Scatter(x=expected_dates, y=expected, mode='lines', name='Expected Balance', line=dict(color='lightblue', dash='dash')))
        
        if actual_dates:
            # Actual Balance (total) - thick green line with markers
            fig.add_trace(go.Scatter(x=actual_dates, y=actual_balances, mode='lines+markers', name='Actual Balance', 
                                   line=dict(color='green', width=4), marker=dict(size=8, color='green')))
            
            # Principal Balance - blue line with markers
            fig.add_trace(go.Scatter(x=actual_dates, y=principal_balances, mode='lines+markers', name='Principal Balance', 
                                   line=dict(color='blue', width=3), marker=dict(size=6, color='blue')))
            
            # Interest Balance - purple line with markers
            if any(i > 0 for i in interest_balances):
                fig.add_trace(go.Scatter(x=actual_dates, y=interest_balances, mode='lines+markers', name='Interest Balance', 
                                       line=dict(color='purple', width=3), marker=dict(size=6, color='purple')))
        
        # Outstanding fines (orange line with markers)
        if actual_dates and any(f > 0 for f in actual_fines):
            fig.add_trace(go.Scatter(x=actual_dates, y=actual_fines, mode='lines+markers', name='Outstanding Fines', 
                                   line=dict(color='orange', width=3), marker=dict(size=6, color='orange')))
        
        # Current time indicator
        all_values = expected + actual_balances + principal_balances + interest_balances + actual_fines
        max_y = max(all_values) if all_values else max(expected)
        fig.add_shape(type='line', x0=current_time, x1=current_time, y0=0, y1=max_y, line=dict(color='red', width=3))
        fig.add_annotation(x=current_time, y=max_y*0.9, text=current_time.strftime('%Y-%m-%d'), bgcolor='red', font=dict(color='white'))
        
        fig.update_layout(title='Loan Progress: Balance Components', height=400, yaxis_title='Amount ($)')
        
        # Debug info
        print(f"Chart data points: {len(actual_dates)}")
        if actual_dates:
            print(f"Date range: {actual_dates[0].strftime('%Y-%m-%d')} to {actual_dates[-1].strftime('%Y-%m-%d')}")
            print(f"Actual Balance range: ${min(actual_balances):,.2f} to ${max(actual_balances):,.2f}")
            print(f"Principal Balance range: ${min(principal_balances):,.2f} to ${max(principal_balances):,.2f}")
            if any(i > 0 for i in interest_balances):
                print(f"Interest Balance range: ${min(interest_balances):,.2f} to ${max(interest_balances):,.2f}")
            if any(f > 0 for f in actual_fines):
                print(f"Fines range: ${min(actual_fines):,.2f} to ${max(actual_fines):,.2f}")
            
            # Show last few data points
            print(f"\nLast data point:")
            print(f"  Actual Balance: ${actual_balances[-1]:,.2f}")
            print(f"  Principal Balance: ${principal_balances[-1]:,.2f}")
            print(f"  Interest Balance: ${interest_balances[-1]:,.2f}")
            print(f"  Fines: ${actual_fines[-1]:,.2f}")
        
        fig.show()

# Callbacks
create_button.on_click(create_loan)
record_button.on_click(record_payment)
time_slider.observe(lambda change: update_display(), names='value')

# Display
display(widgets.VBox([
    widgets.HTML('<h3>🏦 Loan Parameters</h3>'),
    widgets.HBox([principal_input, rate_input, payments_input]),
    widgets.HBox([amortization_input, fine_rate_input]),
    create_button, loan_output
]))

display(widgets.VBox([
    widgets.HTML('<h3>⏰ Time Machine</h3>'),
    time_slider,
    widgets.HBox([payment_input, record_button]),
    balance_output
]))

display(widgets.VBox([
    widgets.HTML('<h3>📊 Chart</h3>'),
    chart_output
]))